### Simply explain Gaussian Mixture Model

A Gaussian mixture model (GMM) is a probabilistic model that assumes all the data points in a dataset are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. In other words, it is a generalization of the k-means clustering algorithm, which is also a special case of GMM, where the number of Gaussian distributions is set to one.

The GMM algorithm is a model-based method for clustering, which means that it uses a generative model to describe the distribution of the data, and it uses the Expectation-Maximization (EM) algorithm to estimate the parameters of the Gaussian distributions from the data. Once the parameters are estimated, the GMM can be used to classify new data points into one of the clusters, or to generate new data points from one of the clusters.

One of the main advantage of Gaussian mixture model, is that it allows for data points to have a probability of belonging to different clusters, which is useful when data points are not clearly separable into distinct clusters. It is commonly used in image segmentation, natural language processing and other fields

In [ ]:
import numpy as np

class GaussianMixtureModel:
    def __init__(self, n_components, max_iter=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.means = None
        self.covariances = None
        self.weights = None
    
    def fit(self, X):
        n_samples, n_features = X.shape
        
        # Initialize means, covariances, and weights
        self.means = np.random.rand(self.n_components, n_features)
        self.covariances = np.array([np.eye(n_features) for _ in range(self.n_components)])
        self.weights = np.ones(self.n_components) / self.n_components
        
        for i in range(self.max_iter):
            # Expectation step
            responsibilities = self._e_step(X)
            
            # Maximization step
            self._m_step(X, responsibilities)
            
            # Check for convergence
            if np.abs(self.weights.sum() - 1) < self.tol:
                break
    
    def predict(self, X):
        responsibilities = self._e_step(X)
        return responsibilities.argmax(axis=1)
    
    def _e_step(self, X):
        n_samples, _ = X.shape
        responsibilities = np.zeros((n_samples, self.n_components))
        for k in range(self.n_components):
            responsibilities[:, k] = self.weights[k] * self._multivariate_gaussian(X, self.means[k], self.covariances[k])
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        return responsibilities
    
    def _m_step(self, X, responsibilities):
        n_samples, n_features = X.shape
        for k in range(self.n_components):
            # Update weights
            self.weights[k] = responsibilities[:, k].sum() / n_samples
            
            # Update means
            weighted_sum = responsibilities[:, k].dot(X)
            self.means[k] = weighted_sum / responsibilities[:, k].sum()
            
            # Update covariances
            centered_data = X - self.means[k]
            weighted_cov = np.dot(centered_data.T, responsibilities[:, k] * centered_data)
            self.covariances[k] = weighted_cov / responsibilities[:, k].sum()
    
    def _multivariate_gaussian(self, X, mean, covariance):
        n_samples = X.shape[0]
        X = X - mean
        return (2 * np.pi) ** (- X.shape[1] / 2) * np.linalg.det(covariance) ** -0.5 * \
            np.exp(-0.5 * np.sum(X @ np.linalg.inv(covariance) * X, axis=1))

